# SC0x Optimization Exercises

### Module 4. Unit 2 | Constrained Optimization

PP1. Jims Meat Packing Company

In [1]:
# import gurobi library
import gurobipy as gp         #Gurobi Python interface
from gurobipy import GRB      #Import as shortcut to avoid writing GP.grb
from itertools import product #product creates the cartesian product from 2 or more lists.

In [2]:
meats = ['LeanMeat','FatMeat']
cuts = ['Chuck','Sirloin']

Meat_options = list(product(meats,cuts))

cost=dict(zip(cuts,[9.3,8.4]))

LeanMeat = {'Chuck':0.09,
            'Sirloin':0.6}
FatMeat={'Chuck':0.02,
            'Sirloin':0.06}#dict(zip(Meat_options,[0.09,0.6,0.02,0.06]))

LeanPerc = dict(zip(meats,[0.3,0.05]))

In [3]:
#defining the model
m=gp.Model('Meats')

Restricted license - for non-production use only - expires 2024-10-28


In [4]:
#Variables
Meats_var = m.addVars(cuts,vtype=GRB.CONTINUOUS,name='Meats')
Meats_var

{'Chuck': <gurobi.Var *Awaiting Model Update*>,
 'Sirloin': <gurobi.Var *Awaiting Model Update*>}

In [5]:
cost

{'Chuck': 9.3, 'Sirloin': 8.4}

In [6]:
#Objective Functin
m.setObjective(Meats_var.prod(cost), GRB.MINIMIZE)

In [7]:
#Constraints
#Lean Meat at least 30%
m.addConstr(Meats_var.prod(LeanMeat) >= LeanPerc['LeanMeat']*Meats_var.sum(),name='LeanMeat%_Const')

<gurobi.Constr *Awaiting Model Update*>

In [8]:
#Fat meat at most 5%
m.addConstr(Meats_var.prod(FatMeat) <= LeanPerc['FatMeat']*Meats_var.sum(),name='FatMeat%_Const')

<gurobi.Constr *Awaiting Model Update*>

In [9]:
#at leats 50 pounds

m.addConstr(Meats_var.sum('*','*') >= 50, name='AtLeast50Pounds')

<gurobi.Constr *Awaiting Model Update*>

In [10]:
#Writting the model
m.write('Meats.lp')

In [11]:
# Details of the model
with open('Meats.lp') as f:
  print (f.read())

\ Model Meats
\ LP format - for model browsing. Use MPS format to capture full model detail.
Minimize
  9.3 Meats[Chuck] + 8.4 Meats[Sirloin]
Subject To
 LeanMeat%_Const: - 0.21 Meats[Chuck] + 0.3 Meats[Sirloin] >= 0
 FatMeat%_Const: - 0.03 Meats[Chuck] + 0.01 Meats[Sirloin] <= 0
 AtLeast50Pounds: Meats[Chuck] + Meats[Sirloin] >= 50
Bounds
End



In [12]:
#Optimize
m.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: AMD Ryzen 7 4800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 3 rows, 2 columns and 6 nonzeros
Model fingerprint: 0x9bc4f007
Coefficient statistics:
  Matrix range     [1e-02, 1e+00]
  Objective range  [8e+00, 9e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+01, 5e+01]
Presolve time: 0.01s
Presolved: 3 rows, 2 columns, 6 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   5.000000e+01   0.000000e+00      0s
       2    4.3125000e+02   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds (0.00 work units)
Optimal objective  4.312500000e+02


In [13]:
#Results
print(f"Optimal objective value: {m.objVal}")

print("\nSites Openned",end = '\n')
for cut in Meats_var:
  if (Meats_var[cut].x>0):
    print(f'Use {Meats_var[cut].x} meat of {cut}')


Optimal objective value: 431.25

Sites Openned
Use 12.499999999999996 meat of Chuck
Use 37.5 meat of Sirloin


PP2. Pasteru Cheese Factory

In [14]:
# import gurobi library
import gurobipy as gp         #Gurobi Python interface
from gurobipy import GRB      #Import as shortcut to avoid writing GP.grb
from itertools import product #product creates the cartesian product from 2 or more lists.

In [15]:
#Data and Variables
Cheeses = ['Original','Cheddar','Wasabi']

Cheese_Price = dict(zip(Cheeses,[20,15,17]))

XRay_time = dict(zip(Cheeses,[8.6,2.2,5.1]))

Weight_time = dict(zip(Cheeses,[7.0,6.8,9.8]))

Prod_Cost = {'XRay_time':0.11,
             'Weight_time':0.22}





In [16]:
#Defining the model
m=gp.Model('CheeseModel')

In [17]:
#Decision Variables
Cheese_var = m.addVars(Cheeses, vtype=GRB.CONTINUOUS, name='Cheeses')
Cheese_var

{'Original': <gurobi.Var *Awaiting Model Update*>,
 'Cheddar': <gurobi.Var *Awaiting Model Update*>,
 'Wasabi': <gurobi.Var *Awaiting Model Update*>}

In [18]:
#Objective Function
m.setObjective(Cheese_var.prod(Cheese_Price)-(Cheese_var.prod(XRay_time)*0.11+Cheese_var.prod(Weight_time)*0.22),GRB.MAXIMIZE)

In [19]:
#Constraints
m.addConstr(Cheese_var.prod(XRay_time)<=480, name='Xray_Machine')

<gurobi.Constr *Awaiting Model Update*>

In [20]:
m.addConstr(Cheese_var.prod(Weight_time)<=540, name='Weight_Machine')

<gurobi.Constr *Awaiting Model Update*>

In [21]:
m.write('Cheese.lp')

In [22]:
# Details of the model
with open('Cheese.lp') as f:
  print (f.read())

\ Model CheeseModel
\ LP format - for model browsing. Use MPS format to capture full model detail.
Maximize
  17.514 Cheeses[Original] + 13.262 Cheeses[Cheddar]
   + 14.283 Cheeses[Wasabi]
Subject To
 Xray_Machine: 8.6 Cheeses[Original] + 2.2 Cheeses[Cheddar]
   + 5.1 Cheeses[Wasabi] <= 480
 Weight_Machine: 7 Cheeses[Original] + 6.8 Cheeses[Cheddar]
   + 9.8 Cheeses[Wasabi] <= 540
Bounds
End



In [23]:
m.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: AMD Ryzen 7 4800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 2 rows, 3 columns and 6 nonzeros
Model fingerprint: 0x77ceb8f1
Coefficient statistics:
  Matrix range     [2e+00, 1e+01]
  Objective range  [1e+01, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+02, 5e+02]
Presolve time: 0.00s
Presolved: 2 rows, 3 columns, 6 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.5059000e+31   4.937500e+30   4.505900e+01      0s
       2    1.2392635e+03   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.239263510e+03


In [24]:
print(f"Optimal objective value: {m.objVal}")

print("\nCheese Produces")
for chis in Cheese_var:
  if (Cheese_var[chis].x>0):
    print(f'Produce {round(Cheese_var[chis].x)} amounts of {chis} Cheese')

Optimal objective value: 1239.2635097493035

Cheese Produces
Produce 48 amounts of Original Cheese
Produce 30 amounts of Cheddar Cheese


## Module 4. Unit 3 | IMILP

 PP2 - BooBoo Magicians  


In [25]:
# import gurobi library
import gurobipy as gp         #Gurobi Python interface
from gurobipy import GRB      #Import as shortcut to avoid writing GP.grb
from itertools import product #product creates the cartesian product from 2 or more lists.

In [26]:
entert_pack = ['Standard','Joyful','Fabulous']

labor_req = dict(zip(entert_pack,[3,5,8]))
Balloon_req = dict(zip(entert_pack,[2,5,8]))
Cost = dict(zip(entert_pack,[2,3,4]))
Price = dict(zip(entert_pack,[3,5,7]))

Part 1.

In [27]:
ballons_inv = 325
labor_avail = 400 

In [28]:
#Defining the model
m = gp.Model('BooBooMagicians')

In [29]:
#Decision Variables
entert_pack_var = m.addVars(entert_pack, vtype=GRB.INTEGER, name= 'EntertainmentPackages')

In [30]:
#Objective Function
m.setObjective(entert_pack_var.prod(Price)-entert_pack_var.prod(Cost), GRB.MAXIMIZE)
entert_pack_var

{'Standard': <gurobi.Var *Awaiting Model Update*>,
 'Joyful': <gurobi.Var *Awaiting Model Update*>,
 'Fabulous': <gurobi.Var *Awaiting Model Update*>}

In [31]:
#Constraints
#labor
m.addConstr((gp.quicksum(entert_pack_var[ent]*labor_req[ent] for ent in entert_pack)<=labor_avail), name= 'LaborAvailable')

<gurobi.Constr *Awaiting Model Update*>

In [32]:
#Balloons
m.addConstr((gp.quicksum(entert_pack_var[ent]*Balloon_req[ent] for ent in entert_pack)<=ballons_inv), name='BaloonAvailable')

<gurobi.Constr *Awaiting Model Update*>

In [33]:
#writting the model
m.write('BooBooMagicians.lp')

In [34]:
# Details of the model
with open('BooBooMagicians.lp') as f:
  print (f.read())

\ Model BooBooMagicians
\ LP format - for model browsing. Use MPS format to capture full model detail.
Maximize
  EntertainmentPackages[Standard] + 2 EntertainmentPackages[Joyful]
   + 3 EntertainmentPackages[Fabulous]
Subject To
 LaborAvailable: 3 EntertainmentPackages[Standard]
   + 5 EntertainmentPackages[Joyful] + 8 EntertainmentPackages[Fabulous]
   <= 400
 BaloonAvailable: 2 EntertainmentPackages[Standard]
   + 5 EntertainmentPackages[Joyful] + 8 EntertainmentPackages[Fabulous]
   <= 325
Bounds
Generals
 EntertainmentPackages[Standard] EntertainmentPackages[Joyful]
 EntertainmentPackages[Fabulous]
End



In [35]:
m.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: AMD Ryzen 7 4800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 2 rows, 3 columns and 6 nonzeros
Model fingerprint: 0x8b0f33c6
Variable types: 0 continuous, 3 integer (0 binary)
Coefficient statistics:
  Matrix range     [2e+00, 8e+00]
  Objective range  [1e+00, 3e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+02, 4e+02]
Found heuristic solution: objective 133.0000000
Presolve time: 0.00s
Presolved: 2 rows, 3 columns, 6 nonzeros
Variable types: 0 continuous, 3 integer (0 binary)

Root relaxation: objective 1.450000e+02, 3 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     145.0000000  145.00000  0.00%     -    0s

Explored

In [36]:
print(f"Optimal objective value: {m.objVal}")

Optimal objective value: 145.0


Part 4

In [37]:
#Additional Information for Part 4
FixCost = dict(zip(entert_pack,[10,5,1]))
ballons_inv = 360
labor_avail = 500 

FixCost

{'Standard': 10, 'Joyful': 5, 'Fabulous': 1}

In [38]:
#Define model 4
m=gp.Model('BooBooMagicians_P4')

In [39]:
#Decision Variables
#Amount of Entrertainment Packages
entert_pack_var = m.addVars(entert_pack, vtype=GRB.INTEGER, name= 'EntertainmentPackages')

In [40]:
fix_cost_var = m.addVars(entert_pack, vtype=GRB.BINARY, name='PackageFixCost')

In [41]:
#Objective Function
m.setObjective(entert_pack_var.prod(Price)-entert_pack_var.prod(Cost)-fix_cost_var.prod(FixCost), GRB.MAXIMIZE)


In [42]:
#Constraints
#Labor
m.addConstr(gp.quicksum(entert_pack_var[ent]*labor_req[ent] for ent in entert_pack)<=labor_avail, name='LaborConstraint')

<gurobi.Constr *Awaiting Model Update*>

In [43]:
#Balloons Available
m.addConstr(gp.quicksum(entert_pack_var[ent]*Balloon_req[ent] for ent in entert_pack)<=ballons_inv, name='BalloonsConstraint')

<gurobi.Constr *Awaiting Model Update*>

In [44]:
#Linking Constraint
m.addConstrs((entert_pack_var[ent]-fix_cost_var[ent]*30_000 <= 0 for ent in entert_pack), name='LinkingConst')

{'Standard': <gurobi.Constr *Awaiting Model Update*>,
 'Joyful': <gurobi.Constr *Awaiting Model Update*>,
 'Fabulous': <gurobi.Constr *Awaiting Model Update*>}

In [45]:
#Write the model
m.write('BooBooMagicians_P4.lp')

In [46]:
# Details of the model
with open('BooBooMagicians_P4.lp') as f:
  print (f.read())

\ Model BooBooMagicians_P4
\ LP format - for model browsing. Use MPS format to capture full model detail.
Maximize
  EntertainmentPackages[Standard] + 2 EntertainmentPackages[Joyful]
   + 3 EntertainmentPackages[Fabulous] - 10 PackageFixCost[Standard]
   - 5 PackageFixCost[Joyful] - PackageFixCost[Fabulous]
Subject To
 LaborConstraint: 3 EntertainmentPackages[Standard]
   + 5 EntertainmentPackages[Joyful] + 8 EntertainmentPackages[Fabulous]
   <= 500
 BalloonsConstraint: 2 EntertainmentPackages[Standard]
   + 5 EntertainmentPackages[Joyful] + 8 EntertainmentPackages[Fabulous]
   <= 360
 LinkingConst[Standard]: EntertainmentPackages[Standard]
   - 30000 PackageFixCost[Standard] <= 0
 LinkingConst[Joyful]: EntertainmentPackages[Joyful]
   - 30000 PackageFixCost[Joyful] <= 0
 LinkingConst[Fabulous]: EntertainmentPackages[Fabulous]
   - 30000 PackageFixCost[Fabulous] <= 0
Bounds
Binaries
 PackageFixCost[Standard] PackageFixCost[Joyful] PackageFixCost[Fabulous]
Generals
 EntertainmentPackag

In [47]:
m.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: AMD Ryzen 7 4800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 5 rows, 6 columns and 12 nonzeros
Model fingerprint: 0xdebfc35f
Variable types: 0 continuous, 6 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+04]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e+02, 5e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 156.0000000
Presolve time: 0.00s
Presolved: 5 rows, 6 columns, 12 nonzeros
Variable types: 0 continuous, 6 integer (3 binary)

Root relaxation: objective 1.624552e+02, 4 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  162.45515    0    2  

In [48]:
print(f"Optimal objective value: {m.objVal}")



Optimal objective value: 159.0


PP1 - Fixed Charge Problem

In [1]:
# import gurobi library
import gurobipy as gp         #Gurobi Python interface
from gurobipy import GRB      #Import as shortcut to avoid writing GP.grb
from itertools import product #product creates the cartesian product from 2 or more lists.

In [2]:
# Data

frames = ['Homer','Turner','Sargent']

labor = dict(zip(frames,[3,2,6]))
glass = dict(zip(frames,[4,3,4]))

max_labor = 150
max_glass = 160

fix_cost = dict(zip(frames,[200,150,100]))

profit = dict(zip(frames,[12-6,8-4,15-8]))


In [3]:
#Initialize model
m = gp.Model('Frames')

Restricted license - for non-production use only - expires 2024-10-28


In [4]:
#Decision Variables
frames_var = m.addVars(frames, vtype=GRB.INTEGER, name='Frames')

In [5]:
frames_bin = m.addVars(frames, vtype=GRB.BINARY, name='Frames_binary')

In [6]:
#Objective Function
m.setObjective(frames_var.prod(profit)-frames_bin.prod(fix_cost),GRB.MAXIMIZE)

In [7]:
#Constraints
#labor
m.addConstr(frames_var.prod(labor)<=max_labor,name='LaborConstr')

<gurobi.Constr *Awaiting Model Update*>

In [8]:
#glass
m.addConstr(frames_var.prod(glass)<=max_glass, name='GlassConstr')

<gurobi.Constr *Awaiting Model Update*>

In [9]:
#linking
m.addConstrs((frames_var[f]-100*frames_bin[f]<=0 for f in frames),name='Linking')

{'Homer': <gurobi.Constr *Awaiting Model Update*>,
 'Turner': <gurobi.Constr *Awaiting Model Update*>,
 'Sargent': <gurobi.Constr *Awaiting Model Update*>}

In [10]:
#write model
m.write('Frames.lp')

In [11]:
# Details of the model
with open('Frames.lp') as f:
  print (f.read())

\ Model Frames
\ LP format - for model browsing. Use MPS format to capture full model detail.
Maximize
  6 Frames[Homer] + 4 Frames[Turner] + 7 Frames[Sargent]
   - 200 Frames_binary[Homer] - 150 Frames_binary[Turner]
   - 100 Frames_binary[Sargent]
Subject To
 LaborConstr: 3 Frames[Homer] + 2 Frames[Turner] + 6 Frames[Sargent]
   <= 150
 GlassConstr: 4 Frames[Homer] + 3 Frames[Turner] + 4 Frames[Sargent]
   <= 160
 Linking[Homer]: Frames[Homer] - 100 Frames_binary[Homer] <= 0
 Linking[Turner]: Frames[Turner] - 100 Frames_binary[Turner] <= 0
 Linking[Sargent]: Frames[Sargent] - 100 Frames_binary[Sargent] <= 0
Bounds
Binaries
 Frames_binary[Homer] Frames_binary[Turner] Frames_binary[Sargent]
Generals
 Frames[Homer] Frames[Turner] Frames[Sargent]
End



In [12]:
#optimize model
m.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: AMD Ryzen 7 4800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 5 rows, 6 columns and 12 nonzeros
Model fingerprint: 0x04382a37
Variable types: 0 continuous, 6 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [4e+00, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+02, 2e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 40.0000000
Presolve time: 0.00s
Presolved: 5 rows, 6 columns, 12 nonzeros
Variable types: 0 continuous, 6 integer (3 binary)

Root relaxation: objective 8.111321e+01, 4 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   81.11321    0    2   

In [13]:
print(f"Optimal objective value: {m.objVal}")

Optimal objective value: 75.0


In [14]:
print("Results of the constraints: (Slack is the difference between the right side of the constraint and the left side)")
for constr in m.getConstrs():
    constr_name = constr.ConstrName
    slack_value = constr.Slack
    print(f"Constraint {constr_name}: Slack Value = {slack_value}")

Results of the constraints: (Slack is the difference between the right side of the constraint and the left side)
Constraint LaborConstr: Slack Value = 0.0
Constraint GlassConstr: Slack Value = 60.0
Constraint Linking[Homer]: Slack Value = 0.0
Constraint Linking[Turner]: Slack Value = 0.0
Constraint Linking[Sargent]: Slack Value = 75.0


## Module 4 - Extra Linear Programming

PP7 - Capital Budgeting Problem

In [60]:
# import gurobi library
import gurobipy as gp         #Gurobi Python interface
from gurobipy import GRB      #Import as shortcut to avoid writing GP.grb
from itertools import product #product creates the cartesian product from 2 or more lists.

In [61]:
#Data
project = [i for i in range(1,6)]

years = ['1st year','2nd year','3rd year']

proyect_year = list(product(project, years))

expenditures = dict(zip(proyect_year,[12,8,4,
                                      7,5,1,
                                      9,6,3,
                                      2,4,8,
                                      4,6,10]))

revenue =  dict(zip(project,[30,20,25,23,27]))

available_fund = dict(zip(years,[25,20,20]))

In [62]:
#Defining the model
m = gp.Model('CapitalBudgeting')

In [63]:
#Decision Variables
project_var = m.addVars(project, vtype=GRB.BINARY, name='Projects')
project_var

{1: <gurobi.Var *Awaiting Model Update*>,
 2: <gurobi.Var *Awaiting Model Update*>,
 3: <gurobi.Var *Awaiting Model Update*>,
 4: <gurobi.Var *Awaiting Model Update*>,
 5: <gurobi.Var *Awaiting Model Update*>}

In [64]:
#Objective Function
m.setObjective(project_var.prod(revenue), GRB.MAXIMIZE)
# - gp.quicksum(project_var[proj]*expenditures[proj, year] for proj, year in proyect_year)

In [65]:
#Constraints
#Available funds
for year in years:
  m.addConstr(gp.quicksum(project_var[proj]*expenditures[proj, year] for proj in project) <=available_fund[year], name='FundsAvailable')

In [66]:
#linking constraint
#for proj in project:
#  m.addConstr(gp.quicksum(project_var[proj]*expenditures[proj,year]- project_var[proj]*expenditures[proj,year] for year in years)  <= 0, name='linkingConstr' )

In [67]:
#at least one project
m.addConstr(project_var.sum('*')>=1,name='AtLeastOneProject')

<gurobi.Constr *Awaiting Model Update*>

In [68]:
m.write('ProjectSelection.lp')

In [69]:
with open('ProjectSelection.lp') as f:
  print (f.read())

\ Model CapitalBudgeting
\ LP format - for model browsing. Use MPS format to capture full model detail.
Maximize
  30 Projects[1] + 20 Projects[2] + 25 Projects[3] + 23 Projects[4]
   + 27 Projects[5]
Subject To
 FundsAvailable: 12 Projects[1] + 7 Projects[2] + 9 Projects[3]
   + 2 Projects[4] + 4 Projects[5] <= 25
 FundsAvailable: 8 Projects[1] + 5 Projects[2] + 6 Projects[3]
   + 4 Projects[4] + 6 Projects[5] <= 20
 FundsAvailable: 4 Projects[1] + Projects[2] + 3 Projects[3]
   + 8 Projects[4] + 10 Projects[5] <= 20
 AtLeastOneProject: Projects[1] + Projects[2] + Projects[3] + Projects[4]
   + Projects[5] >= 1
Bounds
Binaries
 Projects[1] Projects[2] Projects[3] Projects[4] Projects[5]
End



In [70]:
#optimize
m.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: AMD Ryzen 7 4800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 4 rows, 5 columns and 20 nonzeros
Model fingerprint: 0x7ba93007
Variable types: 0 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [2e+01, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Found heuristic solution: objective 73.0000000
Presolve time: 0.00s
Presolved: 4 rows, 5 columns, 19 nonzeros
Variable types: 0 continuous, 5 integer (5 binary)
Found heuristic solution: objective 82.0000000

Root relaxation: cutoff, 3 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0        82.00000   82.00

In [71]:
print(f"Optimal objective value: {m.objVal}")

print("Optimal solution for project_var:")
for proj in project:
    print(f"Project {proj}: Value = {project_var[proj].x}")

Optimal objective value: 82.0
Optimal solution for project_var:
Project 1: Value = 1.0
Project 2: Value = 0.0
Project 3: Value = 1.0
Project 4: Value = 0.0
Project 5: Value = 1.0


PP8 - Machu Pichu Commerce

In [72]:
# import gurobi library
import gurobipy as gp         #Gurobi Python interface
from gurobipy import GRB      #Import as shortcut to avoid writing GP.grb
from itertools import product #product creates the cartesian product from 2 or more lists.

In [73]:
#Data
dc_list = ['DC_1','DC_2','DC_3','DC_4','DC_5']
wholesaler_list = ['WH_1','WH_2','WH_3']

t_cost = dict(zip(dc_list,[0.3,0.2,0.8,0.5,0.4]))
capacity = dict(zip(dc_list,[410,410,320,420,380]))
fixed_cost = dict(zip(dc_list,[80,90,80,90,80]))

dc_to_wh = list(product(dc_list,wholesaler_list))

distance = dict(zip(dc_to_wh,[85,120,90,45,150,85,75,120,95,60,80,90,110,65,95]))

minimum_storage = 900
demand=35

min_dcs = 2
max_dc = 3


In [74]:
distance

{('DC_1', 'WH_1'): 85,
 ('DC_1', 'WH_2'): 120,
 ('DC_1', 'WH_3'): 90,
 ('DC_2', 'WH_1'): 45,
 ('DC_2', 'WH_2'): 150,
 ('DC_2', 'WH_3'): 85,
 ('DC_3', 'WH_1'): 75,
 ('DC_3', 'WH_2'): 120,
 ('DC_3', 'WH_3'): 95,
 ('DC_4', 'WH_1'): 60,
 ('DC_4', 'WH_2'): 80,
 ('DC_4', 'WH_3'): 90,
 ('DC_5', 'WH_1'): 110,
 ('DC_5', 'WH_2'): 65,
 ('DC_5', 'WH_3'): 95}

In [75]:
#Defining the model
m = gp.Model('MachuPichu')

In [76]:
#Decision Variables
DC_var = m.addVars(dc_list,vtype=GRB.BINARY,name='DC')

In [77]:
#Objective
m.setObjective((gp.quicksum(gp.quicksum(DC_var[dc]*distance[dc,wh]*t_cost[dc]*demand for wh in wholesaler_list) + fixed_cost[dc]*DC_var[dc] for dc in dc_list)),GRB.MINIMIZE)

In [78]:
#Constraints
#minimum storage
m.addConstr(DC_var.prod(capacity)>=minimum_storage,name='MinimumStorage')

<gurobi.Constr *Awaiting Model Update*>

In [79]:
#at least 2 DC
m.addConstr(DC_var.sum('*')>=min_dcs, name='AtLeast2DCs')

<gurobi.Constr *Awaiting Model Update*>

In [80]:
#at most 3 DC
m.addConstr(DC_var.sum('*')<=max_dc, name='AtMost3DCs')

<gurobi.Constr *Awaiting Model Update*>

In [81]:
#Write model
m.write('MachuPichu.lp')

In [82]:
#Check model
with open('MachuPichu.lp') as f:
  print (f.read())

\ Model MachuPichu
\ LP format - for model browsing. Use MPS format to capture full model detail.
Minimize
  3177.5 DC[DC_1] + 2050 DC[DC_2] + 8200 DC[DC_3] + 4115 DC[DC_4]
   + 3860 DC[DC_5]
Subject To
 MinimumStorage: 410 DC[DC_1] + 410 DC[DC_2] + 320 DC[DC_3] + 420 DC[DC_4]
   + 380 DC[DC_5] >= 900
 AtLeast2DCs: DC[DC_1] + DC[DC_2] + DC[DC_3] + DC[DC_4] + DC[DC_5] >= 2
 AtMost3DCs: DC[DC_1] + DC[DC_2] + DC[DC_3] + DC[DC_4] + DC[DC_5] <= 3
Bounds
Binaries
 DC[DC_1] DC[DC_2] DC[DC_3] DC[DC_4] DC[DC_5]
End



In [83]:
#Optimize model
m.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: AMD Ryzen 7 4800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 3 rows, 5 columns and 15 nonzeros
Model fingerprint: 0x76e26ee7
Variable types: 0 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+02]
  Objective range  [2e+03, 8e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 9e+02]
Found heuristic solution: objective 13427.500000
Presolve removed 3 rows and 5 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 2: 9087.5 13427.5 

Optimal solution found (tolerance 1.00e-04)
Best objective 9.087500000000e+03, best bound 9.087500000000e+03, gap 0.0000%


In [84]:
print(f"Optimal objective value: {m.objVal}")

print("Optimal DCs:")
for dc in DC_var:
    print(f"DC {dc}: Value = {DC_var[dc].x}")
    
print("Results of the constraints: (Slack is the difference between the right side of the constraint and the left side)")
for constr in m.getConstrs():
    constr_name = constr.ConstrName
    slack_value = constr.Slack
    print(f"Constraint {constr_name}: Slack Value = {slack_value}")
    


Optimal objective value: 9087.5
Optimal DCs:
DC DC_1: Value = 1.0
DC DC_2: Value = 1.0
DC DC_3: Value = 0.0
DC DC_4: Value = 0.0
DC DC_5: Value = 1.0
Results of the constraints: (Slack is the difference between the right side of the constraint and the left side)
Constraint MinimumStorage: Slack Value = -300.0
Constraint AtLeast2DCs: Slack Value = -1.0
Constraint AtMost3DCs: Slack Value = 0.0


PP9 - Railey Motors

In [85]:
# import gurobi library
import gurobipy as gp         #Gurobi Python interface
from gurobipy import GRB      #Import as shortcut to avoid writing GP.grb
from itertools import product #product creates the cartesian product from 2 or more lists.

In [86]:
#Data
plants = ['Ahmedabad','Patna','Hyderabad']
DCs = ['Bhopal','Indore']

routes = list(product(plants,DCs))
demand = dict(zip(DCs,[4800,5700]))
capacity=dict(zip(plants,[4800,3500,2200]))

ship_cost = dict(zip(routes,[16500,10600,12200,12600,10300,9240]))


In [87]:
#Initiate model
m = gp.Model('RaileyMotors')

In [88]:
#Decision Variable
ship_units = m.addVars(routes,vtype=GRB.INTEGER,lb=0,name='ShippingUnits')
ship_units

{('Ahmedabad', 'Bhopal'): <gurobi.Var *Awaiting Model Update*>,
 ('Ahmedabad', 'Indore'): <gurobi.Var *Awaiting Model Update*>,
 ('Patna', 'Bhopal'): <gurobi.Var *Awaiting Model Update*>,
 ('Patna', 'Indore'): <gurobi.Var *Awaiting Model Update*>,
 ('Hyderabad', 'Bhopal'): <gurobi.Var *Awaiting Model Update*>,
 ('Hyderabad', 'Indore'): <gurobi.Var *Awaiting Model Update*>}

In [89]:
#Objective Function
m.setObjective(ship_units.prod(ship_cost), GRB.MINIMIZE)

In [90]:
#Constraints
#Capacity
m.addConstrs(ship_units.sum(p)<=capacity[p] for p in plants)

{'Ahmedabad': <gurobi.Constr *Awaiting Model Update*>,
 'Patna': <gurobi.Constr *Awaiting Model Update*>,
 'Hyderabad': <gurobi.Constr *Awaiting Model Update*>}

In [91]:
#Demand
m.addConstrs(ship_units.sum('*',dc)>=demand[dc] for dc in DCs)

{'Bhopal': <gurobi.Constr *Awaiting Model Update*>,
 'Indore': <gurobi.Constr *Awaiting Model Update*>}

In [92]:
#write model
m.write('RaileyMotors.lp')

In [93]:
# Details of the model
with open('RaileyMotors.lp') as f:
  print (f.read())

\ Model RaileyMotors
\ LP format - for model browsing. Use MPS format to capture full model detail.
Minimize
  16500 ShippingUnits[Ahmedabad,Bhopal]
   + 10600 ShippingUnits[Ahmedabad,Indore]
   + 12200 ShippingUnits[Patna,Bhopal] + 12600 ShippingUnits[Patna,Indore]
   + 10300 ShippingUnits[Hyderabad,Bhopal]
   + 9240 ShippingUnits[Hyderabad,Indore]
Subject To
 R0: ShippingUnits[Ahmedabad,Bhopal] + ShippingUnits[Ahmedabad,Indore]
   <= 4800
 R1: ShippingUnits[Patna,Bhopal] + ShippingUnits[Patna,Indore] <= 3500
 R2: ShippingUnits[Hyderabad,Bhopal] + ShippingUnits[Hyderabad,Indore]
   <= 2200
 R3: ShippingUnits[Ahmedabad,Bhopal] + ShippingUnits[Patna,Bhopal]
   + ShippingUnits[Hyderabad,Bhopal] >= 4800
 R4: ShippingUnits[Ahmedabad,Indore] + ShippingUnits[Patna,Indore]
   + ShippingUnits[Hyderabad,Indore] >= 5700
Bounds
Generals
 ShippingUnits[Ahmedabad,Bhopal] ShippingUnits[Ahmedabad,Indore]
 ShippingUnits[Patna,Bhopal] ShippingUnits[Patna,Indore]
 ShippingUnits[Hyderabad,Bhopal] Shippin

In [94]:
#optimize
m.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: AMD Ryzen 7 4800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 5 rows, 6 columns and 12 nonzeros
Model fingerprint: 0x1fd57f0a
Variable types: 0 continuous, 6 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [9e+03, 2e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+03, 6e+03]
Presolve removed 2 rows and 2 columns
Presolve time: 0.00s
Presolved: 3 rows, 4 columns, 8 nonzeros
Variable types: 0 continuous, 4 integer (0 binary)
Found heuristic solution: objective 1.436232e+08
Found heuristic solution: objective 1.166000e+08

Root relaxation: objective 1.152860e+08, 2 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node T

In [95]:
print(f"Optimal objective value: {m.objVal}")

Optimal objective value: 115286000.0


PP11 - Fancy Furniture

In [1]:
# import gurobi library
import gurobipy as gp         #Gurobi Python interface
from gurobipy import GRB      #Import as shortcut to avoid writing GP.grb
from itertools import product #product creates the cartesian product from 2 or more lists.

In [2]:
#data
prod_facilities = ['C1','T1','F','C2','T2']
wh = ['WH1','WH2']

#new_plants=['C2','T2']

sku_flow = list(product(prod_facilities,wh))

capacity = dict(zip(prod_facilities,[1500,1800,6000,2000,2200]))
demand = dict(zip(wh,[6000,5200]))

fix_cost = dict(zip(['C2','T2'],[41700,21100]))

sku_flow_profit=dict(zip(sku_flow, [82,83,
                                  46,49,
                                  30,35,
                                  86,81,
                                  44,52]))



In [3]:
#Initialize model
m = gp.Model('FancyFurniture')

Restricted license - for non-production use only - expires 2024-10-28


In [4]:
#Decision Variables
sku_flow_var = m.addVars(sku_flow,vtype=GRB.INTEGER,name='sku_flow')
sku_flow_var

{('C1', 'WH1'): <gurobi.Var *Awaiting Model Update*>,
 ('C1', 'WH2'): <gurobi.Var *Awaiting Model Update*>,
 ('T1', 'WH1'): <gurobi.Var *Awaiting Model Update*>,
 ('T1', 'WH2'): <gurobi.Var *Awaiting Model Update*>,
 ('F', 'WH1'): <gurobi.Var *Awaiting Model Update*>,
 ('F', 'WH2'): <gurobi.Var *Awaiting Model Update*>,
 ('C2', 'WH1'): <gurobi.Var *Awaiting Model Update*>,
 ('C2', 'WH2'): <gurobi.Var *Awaiting Model Update*>,
 ('T2', 'WH1'): <gurobi.Var *Awaiting Model Update*>,
 ('T2', 'WH2'): <gurobi.Var *Awaiting Model Update*>}

In [5]:
new_plant_var = m.addVars(fix_cost,vtype=GRB.BINARY,name='NewPlantBinary')
new_plant_var

{'C2': <gurobi.Var *Awaiting Model Update*>,
 'T2': <gurobi.Var *Awaiting Model Update*>}

In [6]:
#Set Objective
m.setObjective(sku_flow_var.prod(sku_flow_profit) - new_plant_var.prod(fix_cost),GRB.MAXIMIZE)

In [7]:
#Constraints
#demand
m.addConstrs((sku_flow_var.sum('*',w)==demand[w] for w in wh),name='DemandConstr')

{'WH1': <gurobi.Constr *Awaiting Model Update*>,
 'WH2': <gurobi.Constr *Awaiting Model Update*>}

In [8]:
#capacity
m.addConstrs((sku_flow_var.sum(pf)<=capacity[pf] for pf in prod_facilities),name = 'CapacityConstr')

{'C1': <gurobi.Constr *Awaiting Model Update*>,
 'T1': <gurobi.Constr *Awaiting Model Update*>,
 'F': <gurobi.Constr *Awaiting Model Update*>,
 'C2': <gurobi.Constr *Awaiting Model Update*>,
 'T2': <gurobi.Constr *Awaiting Model Update*>}

In [9]:
#only open one
m.addConstr(new_plant_var.sum()==1,name='Only1NewPlant')

<gurobi.Constr *Awaiting Model Update*>

In [10]:
#linking constraint
m.addConstrs((sku_flow_var.sum(pf)-10_000*new_plant_var.sum(pf) <=0 for pf in prod_facilities[-2:]),name='LinkingConstrs')

{'C2': <gurobi.Constr *Awaiting Model Update*>,
 'T2': <gurobi.Constr *Awaiting Model Update*>}

In [11]:
#write model
m.write('FancyFurniture.lp')

In [12]:
# Details of the model
with open('FancyFurniture.lp') as f:
  print (f.read())

\ Model FancyFurniture
\ LP format - for model browsing. Use MPS format to capture full model detail.
Maximize
  82 sku_flow[C1,WH1] + 83 sku_flow[C1,WH2] + 46 sku_flow[T1,WH1]
   + 49 sku_flow[T1,WH2] + 30 sku_flow[F,WH1] + 35 sku_flow[F,WH2]
   + 86 sku_flow[C2,WH1] + 81 sku_flow[C2,WH2] + 44 sku_flow[T2,WH1]
   + 52 sku_flow[T2,WH2] - 41700 NewPlantBinary[C2]
   - 21100 NewPlantBinary[T2]
Subject To
 DemandConstr[WH1]: sku_flow[C1,WH1] + sku_flow[T1,WH1] + sku_flow[F,WH1]
   + sku_flow[C2,WH1] + sku_flow[T2,WH1] = 6000
 DemandConstr[WH2]: sku_flow[C1,WH2] + sku_flow[T1,WH2] + sku_flow[F,WH2]
   + sku_flow[C2,WH2] + sku_flow[T2,WH2] = 5200
 CapacityConstr[C1]: sku_flow[C1,WH1] + sku_flow[C1,WH2] <= 1500
 CapacityConstr[T1]: sku_flow[T1,WH1] + sku_flow[T1,WH2] <= 1800
 CapacityConstr[F]: sku_flow[F,WH1] + sku_flow[F,WH2] <= 6000
 CapacityConstr[C2]: sku_flow[C2,WH1] + sku_flow[C2,WH2] <= 2000
 CapacityConstr[T2]: sku_flow[T2,WH1] + sku_flow[T2,WH2] <= 2200
 Only1NewPlant: NewPlantBina

In [13]:
#optimize model
m.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: AMD Ryzen 7 4800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 10 rows, 12 columns and 28 nonzeros
Model fingerprint: 0xff7114b5
Variable types: 0 continuous, 12 integer (2 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [3e+01, 4e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+03]
Found heuristic solution: objective 529500.00000
Presolve removed 3 rows and 1 columns
Presolve time: 0.00s
Presolved: 7 rows, 11 columns, 22 nonzeros
Variable types: 0 continuous, 11 integer (1 binary)

Root relaxation: objective 5.391000e+05, 9 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    539100

In [14]:
print(f"Optimal objective value: {m.objVal}")

Optimal objective value: 539100.0


In [15]:
print("Results of the constraints: (Slack is the difference between the right side of the constraint and the left side)")
for constr in m.getConstrs():
    constr_name = constr.ConstrName
    slack_value = constr.Slack
    print(f"Constraint {constr_name}: Slack Value = {slack_value}")

Results of the constraints: (Slack is the difference between the right side of the constraint and the left side)
Constraint DemandConstr[WH1]: Slack Value = 0.0
Constraint DemandConstr[WH2]: Slack Value = 0.0
Constraint CapacityConstr[C1]: Slack Value = 0.0
Constraint CapacityConstr[T1]: Slack Value = 0.0
Constraint CapacityConstr[F]: Slack Value = 100.0
Constraint CapacityConstr[C2]: Slack Value = 0.0
Constraint CapacityConstr[T2]: Slack Value = 2200.0
Constraint Only1NewPlant: Slack Value = 0.0
Constraint LinkingConstrs[C2]: Slack Value = 8000.0
Constraint LinkingConstrs[T2]: Slack Value = 0.0


## Module 4. Unit 4 - Network NLP

PP1 - Transportation Problem

In [96]:
# import gurobi library
import gurobipy as gp         #Gurobi Python interface
from gurobipy import GRB      #Import as shortcut to avoid writing GP.grb
from itertools import product #product creates the cartesian product from 2 or more lists.

In [97]:
#data
facilities=['Chicago','Atlanta','Denver']
destination=['Boston','Seattle','Tampa']

routes = list(product(facilities,destination))

demand = dict(zip(destination,[11000,6300,7400]))
capacity = dict(zip(facilities,[10000]*3))

ship_cost = dict(zip(routes,[1.04,1.27,1.22,
                             1.23,1.93,0.6,
                             1.92,0.94,1.03]))



In [98]:
# Initialize model
m = gp.Model('GreekYogurt')

In [99]:
# Decision Variables
ship_units = m.addVars(routes, vtype=GRB.INTEGER,lb=0,name='UnitsShipped' )

In [100]:
#Objective Function
m.setObjective(ship_units.prod(ship_cost), GRB.MINIMIZE)

In [101]:
#Constraints
#capacity
m.addConstrs((ship_units.sum(fac) <=capacity[fac] for fac in facilities), name= 'CapacityConstr')

{'Chicago': <gurobi.Constr *Awaiting Model Update*>,
 'Atlanta': <gurobi.Constr *Awaiting Model Update*>,
 'Denver': <gurobi.Constr *Awaiting Model Update*>}

In [102]:
#demand
m.addConstrs((ship_units.sum('*',dest)>=demand[dest] for dest in destination), name='DemandConstr')

{'Boston': <gurobi.Constr *Awaiting Model Update*>,
 'Seattle': <gurobi.Constr *Awaiting Model Update*>,
 'Tampa': <gurobi.Constr *Awaiting Model Update*>}

In [103]:
#write model
m.write('GreekYogurt.lp')

In [104]:
# Details of the model
with open('GreekYogurt.lp') as f:
  print (f.read())

\ Model GreekYogurt
\ LP format - for model browsing. Use MPS format to capture full model detail.
Minimize
  1.04 UnitsShipped[Chicago,Boston] + 1.27 UnitsShipped[Chicago,Seattle]
   + 1.22 UnitsShipped[Chicago,Tampa] + 1.23 UnitsShipped[Atlanta,Boston]
   + 1.93 UnitsShipped[Atlanta,Seattle] + 0.6 UnitsShipped[Atlanta,Tampa]
   + 1.92 UnitsShipped[Denver,Boston] + 0.94 UnitsShipped[Denver,Seattle]
   + 1.03 UnitsShipped[Denver,Tampa]
Subject To
 CapacityConstr[Chicago]: UnitsShipped[Chicago,Boston]
   + UnitsShipped[Chicago,Seattle] + UnitsShipped[Chicago,Tampa] <= 10000
 CapacityConstr[Atlanta]: UnitsShipped[Atlanta,Boston]
   + UnitsShipped[Atlanta,Seattle] + UnitsShipped[Atlanta,Tampa] <= 10000
 CapacityConstr[Denver]: UnitsShipped[Denver,Boston]
   + UnitsShipped[Denver,Seattle] + UnitsShipped[Denver,Tampa] <= 10000
 DemandConstr[Boston]: UnitsShipped[Chicago,Boston]
   + UnitsShipped[Atlanta,Boston] + UnitsShipped[Denver,Boston] >= 11000
 DemandConstr[Seattle]: UnitsShipped[Chic

In [105]:
m.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: AMD Ryzen 7 4800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 6 rows, 9 columns and 18 nonzeros
Model fingerprint: 0xc5668a60
Variable types: 0 continuous, 9 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e-01, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+03, 1e+04]
Presolve time: 0.00s
Presolved: 6 rows, 9 columns, 18 nonzeros
Variable types: 0 continuous, 9 integer (0 binary)
Found heuristic solution: objective 25550.930000
Found heuristic solution: objective 23582.570000

Root relaxation: objective 2.199200e+04, 3 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    

In [106]:
print(f"Optimal objective value: {m.objVal}")

Optimal objective value: 21992.0


PP2 - Transshipment Problem  

'Ralph Calvin Hilfiger'

In [1]:
# import gurobi library
import gurobipy as gp         #Gurobi Python interface
from gurobipy import GRB      #Import as shortcut to avoid writing GP.grb
from itertools import product #product creates the cartesian product from 2 or more lists.

In [2]:
factory = ['Factory1','Factory2','Factory3','Factory4','Factory5']
crossdock = ['CR1','CR2']
DC = ['DC1','DC2','DC3','DC4','DC5']

capacity = dict(zip(factory,[200,300,100,150,220]))

demand = dict(zip(DC,[150,100,110,200,180]))

fact2CR = list(product(crossdock,factory))
CR2DC = list(product(crossdock,DC))

fact2CR_cost = dict(zip(fact2CR,[30,23,35,70,65,
                                 50,66,14,12,70]))

CR2DC_cost = dict(zip(CR2DC,[12,25,22,40,41,
                             65,22,23,12,15]))

In [3]:
#Initialize model
m = gp.Model('Ralph')

Restricted license - for non-production use only - expires 2024-10-28


In [4]:
#Decision Variable
fact2CR_var = m.addVars(fact2CR,vtype=GRB.INTEGER,name='Factory_2_CR')
fact2CR_var

{('CR1', 'Factory1'): <gurobi.Var *Awaiting Model Update*>,
 ('CR1', 'Factory2'): <gurobi.Var *Awaiting Model Update*>,
 ('CR1', 'Factory3'): <gurobi.Var *Awaiting Model Update*>,
 ('CR1', 'Factory4'): <gurobi.Var *Awaiting Model Update*>,
 ('CR1', 'Factory5'): <gurobi.Var *Awaiting Model Update*>,
 ('CR2', 'Factory1'): <gurobi.Var *Awaiting Model Update*>,
 ('CR2', 'Factory2'): <gurobi.Var *Awaiting Model Update*>,
 ('CR2', 'Factory3'): <gurobi.Var *Awaiting Model Update*>,
 ('CR2', 'Factory4'): <gurobi.Var *Awaiting Model Update*>,
 ('CR2', 'Factory5'): <gurobi.Var *Awaiting Model Update*>}

In [5]:
CR2DC_var = m.addVars(CR2DC, vtype=GRB.INTEGER,name='CR_2_DC')

In [6]:
#Set Objective Function
m.setObjective(fact2CR_var.prod(fact2CR_cost)+CR2DC_var.prod(CR2DC_cost),GRB.MINIMIZE)

In [7]:
#Constraints
#Capacity
m.addConstrs((fact2CR_var.sum('*',f)<=capacity[f] for f in factory), name='CapacityConst')

{'Factory1': <gurobi.Constr *Awaiting Model Update*>,
 'Factory2': <gurobi.Constr *Awaiting Model Update*>,
 'Factory3': <gurobi.Constr *Awaiting Model Update*>,
 'Factory4': <gurobi.Constr *Awaiting Model Update*>,
 'Factory5': <gurobi.Constr *Awaiting Model Update*>}

In [8]:
#Demand
m.addConstrs((CR2DC_var.sum('*',dc)>=demand[dc] for dc in DC),name='DemandConstr')

{'DC1': <gurobi.Constr *Awaiting Model Update*>,
 'DC2': <gurobi.Constr *Awaiting Model Update*>,
 'DC3': <gurobi.Constr *Awaiting Model Update*>,
 'DC4': <gurobi.Constr *Awaiting Model Update*>,
 'DC5': <gurobi.Constr *Awaiting Model Update*>}

In [9]:
#Conservation of flow
m.addConstrs((fact2CR_var.sum(cr)-CR2DC_var.sum(cr)==0 for cr in crossdock),name='Conservation of flow')

{'CR1': <gurobi.Constr *Awaiting Model Update*>,
 'CR2': <gurobi.Constr *Awaiting Model Update*>}

In [10]:
#write model
m.write('Ralph.lp')

In [11]:
# Details of the model
with open('Ralph.lp') as f:
  print (f.read())

\ Model Ralph
\ LP format - for model browsing. Use MPS format to capture full model detail.
Minimize
  30 Factory_2_CR[CR1,Factory1] + 23 Factory_2_CR[CR1,Factory2]
   + 35 Factory_2_CR[CR1,Factory3] + 70 Factory_2_CR[CR1,Factory4]
   + 65 Factory_2_CR[CR1,Factory5] + 50 Factory_2_CR[CR2,Factory1]
   + 66 Factory_2_CR[CR2,Factory2] + 14 Factory_2_CR[CR2,Factory3]
   + 12 Factory_2_CR[CR2,Factory4] + 70 Factory_2_CR[CR2,Factory5]
   + 12 CR_2_DC[CR1,DC1] + 25 CR_2_DC[CR1,DC2] + 22 CR_2_DC[CR1,DC3]
   + 40 CR_2_DC[CR1,DC4] + 41 CR_2_DC[CR1,DC5] + 65 CR_2_DC[CR2,DC1]
   + 22 CR_2_DC[CR2,DC2] + 23 CR_2_DC[CR2,DC3] + 12 CR_2_DC[CR2,DC4]
   + 15 CR_2_DC[CR2,DC5]
Subject To
 CapacityConst[Factory1]: Factory_2_CR[CR1,Factory1]
   + Factory_2_CR[CR2,Factory1] <= 200
 CapacityConst[Factory2]: Factory_2_CR[CR1,Factory2]
   + Factory_2_CR[CR2,Factory2] <= 300
 CapacityConst[Factory3]: Factory_2_CR[CR1,Factory3]
   + Factory_2_CR[CR2,Factory3] <= 100
 CapacityConst[Factory4]: Factory_2_CR[CR1,Fact

In [12]:
#optimize model
m.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: AMD Ryzen 7 4800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 12 rows, 20 columns and 40 nonzeros
Model fingerprint: 0x674e5a3e
Variable types: 0 continuous, 20 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 7e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 3e+02]
Found heuristic solution: objective 50960.000000
Presolve time: 0.00s
Presolved: 12 rows, 20 columns, 40 nonzeros
Variable types: 0 continuous, 20 integer (0 binary)
Found heuristic solution: objective 50899.000000

Root relaxation: objective 3.022000e+04, 8 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0              

In [13]:
print(f"Optimal objective value: {m.objVal}")


Optimal objective value: 30220.0


In [14]:
print("Results of the constraints: (Slack is the difference between the right side of the constraint and the left side)")
for constr in m.getConstrs():
    constr_name = constr.ConstrName
    slack_value = constr.Slack
    print(f"Constraint {constr_name}: Slack Value = {slack_value}")

Results of the constraints: (Slack is the difference between the right side of the constraint and the left side)
Constraint CapacityConst[Factory1]: Slack Value = 10.0
Constraint CapacityConst[Factory2]: Slack Value = 0.0
Constraint CapacityConst[Factory3]: Slack Value = 0.0
Constraint CapacityConst[Factory4]: Slack Value = 0.0
Constraint CapacityConst[Factory5]: Slack Value = 220.0
Constraint DemandConstr[DC1]: Slack Value = -0.0
Constraint DemandConstr[DC2]: Slack Value = -0.0
Constraint DemandConstr[DC3]: Slack Value = -0.0
Constraint DemandConstr[DC4]: Slack Value = -0.0
Constraint DemandConstr[DC5]: Slack Value = -0.0
Constraint Conservation of flow[CR1]: Slack Value = 0.0
Constraint Conservation of flow[CR2]: Slack Value = 0.0


## Module 4 | Test
Problem 2: Production for softdrinks


In [107]:
# import gurobi library
import gurobipy as gp         #Gurobi Python interface
from gurobipy import GRB      #Import as shortcut to avoid writing GP.grb
from itertools import product #product creates the cartesian product from 2 or more lists.

In [108]:
#data and variables
drinks = ['Lemon','Orange']

water = dict(zip(drinks,[54,80]))
sugar = dict(zip(drinks,[24,13]))
vitamin = dict(zip(drinks,[24,15]))

inventory = {'water':9800,
             'sugar':2980,
             'vitamin':2080}

drink_price=dict(zip(drinks,[60,71]))

In [109]:
#Defining the model
m = gp.Model('softdrinks')

In [110]:
#Decision Variables
Drinks_var = m.addVars(drinks,vtype=GRB.CONTINUOUS,name='Drinks')

In [111]:
#Objective Function
m.setObjective(Drinks_var.prod(drink_price), GRB.MAXIMIZE)

In [112]:
#Constraints
#Water
m.addConstr(Drinks_var.prod(water)<=inventory['water'], name='WaterConstr')

<gurobi.Constr *Awaiting Model Update*>

In [113]:
#Sugar
m.addConstr(Drinks_var.prod(sugar)<=inventory['sugar'],name='SugarConstr')

<gurobi.Constr *Awaiting Model Update*>

In [114]:
#Vitamin
m.addConstr(Drinks_var.prod(vitamin)<=inventory['vitamin'],name='VitaminConstr')

<gurobi.Constr *Awaiting Model Update*>

In [115]:
#Writting the model
m.write('Drinks.lp')

# Details of the model
with open('Drinks.lp') as f:
  print (f.read())

\ Model softdrinks
\ LP format - for model browsing. Use MPS format to capture full model detail.
Maximize
  60 Drinks[Lemon] + 71 Drinks[Orange]
Subject To
 WaterConstr: 54 Drinks[Lemon] + 80 Drinks[Orange] <= 9800
 SugarConstr: 24 Drinks[Lemon] + 13 Drinks[Orange] <= 2980
 VitaminConstr: 24 Drinks[Lemon] + 15 Drinks[Orange] <= 2080
Bounds
End



In [116]:
#Optimize
m.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: AMD Ryzen 7 4800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 3 rows, 2 columns and 6 nonzeros
Model fingerprint: 0xdfa8c07c
Coefficient statistics:
  Matrix range     [1e+01, 8e+01]
  Objective range  [6e+01, 7e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+03, 1e+04]
Presolve time: 0.00s
Presolved: 3 rows, 2 columns, 6 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.1937500e+31   3.406250e+30   1.193750e+01      0s
       2    8.9085405e+03   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds (0.00 work units)
Optimal objective  8.908540541e+03


In [117]:
print(f"Optimal objective value: {m.objVal}")

print("\nDrinks Produces")
for d in Drinks_var:
  if (Drinks_var[d].x>0):
    print(f'Produced {round(Drinks_var[d].x)} amounts of {d}')

Optimal objective value: 8908.540540540542

Drinks Produces
Produced 17 amounts of Lemon
Produced 111 amounts of Orange


Problem 3

In [1]:
# import gurobi library
import gurobipy as gp         #Gurobi Python interface
from gurobipy import GRB      #Import as shortcut to avoid writing GP.grb
from itertools import product #product creates the cartesian product from 2 or more lists.

In [16]:
#Data
market = ['M1','M2','M3']
roasting_plant = ['N','S']
DCs = ['A','B']

demand = dict(zip(market,[500,500,350]))
capacity = dict(zip(roasting_plant,[1000,350]))

plant_2_DC = list(product(DCs, roasting_plant))
DC_2_Market = list(product(DCs,market))

plant_2_DC_cost = dict(zip(plant_2_DC,[51,57,
                                       59,53]))

DC_2_Market_cost = dict(zip(DC_2_Market,[73,62,53,
                                         56,73,64]))

DC_2_Market_cost


{('A', 'M1'): 73,
 ('A', 'M2'): 62,
 ('A', 'M3'): 53,
 ('B', 'M1'): 56,
 ('B', 'M2'): 73,
 ('B', 'M3'): 64}

In [17]:
#Initialize model
m = gp.Model('PureCoffee')

In [18]:
#Decision Variables
plant_2_DC_var = m.addVars(plant_2_DC,vtype=GRB.INTEGER,name='plant_2_dc')

DC_2_Market_var = m.addVars(DC_2_Market,vtype=GRB.INTEGER,name='DC_2_Market')

In [19]:
#Set Objective Function
m.setObjective(plant_2_DC_var.prod(plant_2_DC_cost)+DC_2_Market_var.prod(DC_2_Market_cost),GRB.MINIMIZE)

In [20]:
#Constraints
#Demand
m.addConstrs((DC_2_Market_var.sum('*',m)>=demand[m] for m in market), name='DemandConstr')

{'M1': <gurobi.Constr *Awaiting Model Update*>,
 'M2': <gurobi.Constr *Awaiting Model Update*>,
 'M3': <gurobi.Constr *Awaiting Model Update*>}

In [21]:
#Capacity
m.addConstrs((plant_2_DC_var.sum('*',p)<=capacity[p] for p in roasting_plant),name='CapacityConstr')

{'N': <gurobi.Constr *Awaiting Model Update*>,
 'S': <gurobi.Constr *Awaiting Model Update*>}

In [22]:
#Conservation of flow
m.addConstrs((DC_2_Market_var.sum(dc) - plant_2_DC_var.sum(dc) == 0 for dc in DCs), name='ConservationOfFlow')

{'A': <gurobi.Constr *Awaiting Model Update*>,
 'B': <gurobi.Constr *Awaiting Model Update*>}

In [23]:
#write model
m.write('PureCoffee.lp')

In [24]:
# Details of the model
with open('PureCoffee.lp') as f:
  print (f.read())

\ Model PureCoffee
\ LP format - for model browsing. Use MPS format to capture full model detail.
Minimize
  51 plant_2_dc[A,N] + 57 plant_2_dc[A,S] + 59 plant_2_dc[B,N]
   + 53 plant_2_dc[B,S] + 73 DC_2_Market[A,M1] + 62 DC_2_Market[A,M2]
   + 53 DC_2_Market[A,M3] + 56 DC_2_Market[B,M1] + 73 DC_2_Market[B,M2]
   + 64 DC_2_Market[B,M3]
Subject To
 DemandConstr[M1]: DC_2_Market[A,M1] + DC_2_Market[B,M1] >= 500
 DemandConstr[M2]: DC_2_Market[A,M2] + DC_2_Market[B,M2] >= 500
 DemandConstr[M3]: DC_2_Market[A,M3] + DC_2_Market[B,M3] >= 350
 CapacityConstr[N]: plant_2_dc[A,N] + plant_2_dc[B,N] <= 1000
 CapacityConstr[S]: plant_2_dc[A,S] + plant_2_dc[B,S] <= 350
 ConservationOfFlow[A]: - plant_2_dc[A,N] - plant_2_dc[A,S]
   + DC_2_Market[A,M1] + DC_2_Market[A,M2] + DC_2_Market[A,M3] = 0
 ConservationOfFlow[B]: - plant_2_dc[B,N] - plant_2_dc[B,S]
   + DC_2_Market[B,M1] + DC_2_Market[B,M2] + DC_2_Market[B,M3] = 0
Bounds
Generals
 plant_2_dc[A,N] plant_2_dc[A,S] plant_2_dc[B,N] plant_2_dc[B,S]
 

In [25]:
#optimize model
m.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: AMD Ryzen 7 4800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 7 rows, 10 columns and 20 nonzeros
Model fingerprint: 0x6bc3551b
Variable types: 0 continuous, 10 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+01, 7e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+02, 1e+03]
Presolve time: 0.00s
Presolved: 7 rows, 10 columns, 20 nonzeros
Variable types: 0 continuous, 10 integer (0 binary)

Root relaxation: objective 1.483000e+05, 7 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    148300.00000 148300.000  0.00%     -    0s

Explored 1 nodes (7 simplex iterations) in 0.01 se

In [26]:
print(f"Optimal objective value: {m.objVal}")

Optimal objective value: 148300.0


In [27]:
print("Results of the constraints: (Slack is the difference between the right side of the constraint and the left side)")
for constr in m.getConstrs():
    constr_name = constr.ConstrName
    slack_value = constr.Slack
    print(f"Constraint {constr_name}: Slack Value = {slack_value}")

Results of the constraints: (Slack is the difference between the right side of the constraint and the left side)
Constraint DemandConstr[M1]: Slack Value = -0.0
Constraint DemandConstr[M2]: Slack Value = -0.0
Constraint DemandConstr[M3]: Slack Value = -0.0
Constraint CapacityConstr[N]: Slack Value = 0.0
Constraint CapacityConstr[S]: Slack Value = 0.0
Constraint ConservationOfFlow[A]: Slack Value = 0.0
Constraint ConservationOfFlow[B]: Slack Value = 0.0
